# **ACA** - **AADIOS**

This is a notebook containing examples for the talk *"Difference-differential polynomials in SageMath"* in the session **Algebraic and Algorithmic Aspects of Differential and Integral Operators** (AADIOS) within the context of the *28th International Conference on Applications of Computer Algebra* (ACA'2023), July 17--21, 2023, Warsaw.

In [1]:
import sys; sys.path.insert(0, "..") # dalgebra is here
%display latex
from dalgebra import *

## **1. Creation of Difference-Differential Polynomials**

### 1.1 Creation of base rings

Differential rings:

In [2]:
DQ_x = DifferentialRing(QQ[x], [1]) # DQ_x = (Q[x], dx)
DQ_ex = DifferentialRing(QQ['e_x'], ['e_x']) # DQ_ex = (Q[e^x], e^xde^x)

DQ_x, DQ_ex

(Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)],
 Differential Ring [[Univariate Polynomial Ring in e_x over Rational Field], (e_x*d/de_x,)])

Difference rings:

In [3]:
SQ_x = DifferenceRing(QQ[x], [x+1]) # SQ_x = (Q[x], s: x -> x+1)
SQ_ex = DifferenceRing(QQ['e', 'e_x'], ['e', 'e*e_x'])# QQ['e', 'e_x'].Hom(QQ['e', 'e_x'])(['e','e*e_x']))
SQ_x, SQ_ex

(Difference Ring [[Univariate Polynomial Ring in x over Rational Field], (Hom({x: x + 1}),)],
 Difference Ring [[Multivariate Polynomial Ring in e, e_x over Rational Field], (Hom({e: e, e_x: e*e_x}),)])

Mixed:

In [4]:
SDQ_x = DifferenceRing(DQ_x, [x+1]); # SDQ_x = (Q[x], (dx, s: x -> x+1))
SDQ_ex = DRing(QQ['x','e','e_x'], [1, 0, 'e_x'], ['x+1', 'e', 'e*e_x'], types=["derivation", "homomorphism"])
SDQ_x, SDQ_ex

(Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx, Hom({x: x + 1}))],
 Ring [[Multivariate Polynomial Ring in x, e, e_x over Rational Field], (d/dx + e_x*d/de_x, Hom({x: x + 1, e: e, e_x: e*e_x}))])

In [5]:
x, e, e_x = SDQ_ex.gens()
el = (x^2 - e^2)/e_x
show("element: ", el)
show("derivative: ", el.derivative())
show("shift: ", el.shift())

'element: ' (x^2 - e^2)/e_x

'derivative: ' (-x^2*e + e^3 + x^2 - e^2 + 2*x + 1)/e_x

'shift: ' (x^2 - e^2 + 2*x + 1)/(e*e_x)

### 1.2 Creation of difference-differential polynomials

#### Differential ring of polynomials

In [6]:
DQ_x_uv.<u,v> = DPolynomialRing(DQ_x); x = DQ_x('x')
DQ_x_uv

Ring of operator polynomials in (u, v) over Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)]

In [7]:
## It inherits the derivative over DQ_x
(u + x*u[1]).derivative()

x*u_2 + 2*u_1

In [8]:
## Derivating simply increases the index
u[0].derivative(times=20)

u_20

In [9]:
## Leibniz rule
(u*v).derivative() == (u*v[1] + v*u[1])

True

#### Difference ring of polynomials

In [10]:
SQ_ex_yz.<y,z> = DPolynomialRing(SQ_ex); e, e_x = SQ_ex.gens()
SQ_ex_yz

Ring of operator polynomials in (y, z) over Difference Ring [[Multivariate Polynomial Ring in e, e_x over Rational Field], (Hom({e: e, e_x: e*e_x}),)]

In [11]:
## It inherits the shift over DQ_x
(e*y[0] + z[1]*e_x).shift()

e*y_1 + e*e_x*z_2

In [12]:
## Computing the shift simply increases the index
z[0].shift(times=20)

z_20

In [13]:
## The shift is a ring homomorphism
(y*z).shift() == y[1]*z[1]

True

#### Difference-Differential polynomials

In [14]:
SDQ_ex_uv.<u,v> = DPolynomialRing(SDQ_ex); x, e, e_x = SDQ_ex.gens()
SDQ_ex_uv

Ring of operator polynomials in (u, v) over Ring [[Multivariate Polynomial Ring in x, e, e_x over Rational Field], (d/dx + e_x*d/de_x, Hom({x: x + 1, e: e, e_x: e*e_x}))]

In [15]:
# Now we have 2 indices: one for each operator
u[0,0]

u_0_0

In [16]:
# Applying each operation in any order just increase the indices accordingly
u[0,0].derivative(), u[0,0].shift()

(u_1_0, u_0_1)

In [17]:
u[0,0].derivative(times=5).shift(times=5).derivative(times=2)

u_7_5

In [18]:
## Now, all properties are inherited
element = u*e_x + v*e*x
show("element: ", element)
show("derivative: ", element.derivative())
show("shift: ", element.shift())
show("der-shift: ", element.derivative().shift())

'element: ' e_x*u_0_0 + x*e*v_0_0

'derivative: ' e_x*u_1_0 + e_x*u_0_0 + x*e*v_1_0 + e*v_0_0

'shift: ' e*e_x*u_0_1 + (x*e + e)*v_0_1

'der-shift: ' e*e_x*u_1_1 + e*e_x*u_0_1 + (x*e + e)*v_1_1 + e*v_0_1

## **2. Systems of difference/differential polynomials.**

### 2.1 Creation of the systems: determining important variables

In [10]:
# Creating ring of differential polynomials
R.<t,y,z> = DifferentialPolynomialRing(QQ) 
equations = [
    t[1] - t^2 - 1, # t(x) = tan(x)
    -y[2]*y[0] + y[1]^2 + y[1]*y[0], # y(x) = e^(e^x -1)
    z[0] - t[0] - y[0]
]
equations

[-t_0^2 + t_1 - 1, -y_2*y_0 + y_1^2 + y_1*y_0, -t_0 - y_0 + z_0]

In [12]:
system = DSystem(equations, variables=[t,y]); system

System over [Ring of operator polynomials in (t, y, z) over Differential Ring [[Rational Field], (0,)]] with variables [(t_*, y_*)]:
{
	-t_0^2 + t_1 - 1 == 0
	-y_2*y_0 + y_1^2 + y_1*y_0 == 0
	-t_0 - y_0 + z_0 == 0
}

### 2.1 Eliminating variables: Difference resultants and Differential resultants

In [14]:
import logging
logger.setLevel(logging.INFO)
res = system.diff_resultant()

2023-07-17 22:00:05 INFO     We could not decide a better algorithm: using iterative elimination
2023-07-17 22:00:05 INFO     Checking if there is any linear variable...
2023-07-17 22:00:05 INFO     No linear variable remain in the system. We proceed by univariate eliminations
2023-07-17 22:00:05 INFO     Several eliminations are needed --> we use recursion
2023-07-17 22:00:05 INFO     Picking the best variable to start with...
2023-07-17 22:00:05 INFO     Picked differential variable t_*
2023-07-17 22:00:05 INFO     Picked the pivot [-t_0 - y_0 + z_0...] for differential elimination
2023-07-17 22:00:05 INFO     Computing the elimination for all pair of equations...
2023-07-17 22:00:05 INFO     We could not decide a better algorithm: using iterative elimination
2023-07-17 22:00:05 INFO     Checking if there is any linear variable...
2023-07-17 22:00:05 INFO     No linear variable remain in the system. We proceed by univariate eliminations
2023-07-17 22:00:05 INFO     Only one variable 

In [17]:
len(res.monomials())

499

In [21]:
QQ[res.variables()](res).factor()

(z_0^2 - z_1 + 1) * (432*z_1^8*z_0^2 - 1728*z_2*z_1^6*z_0^3 + 1728*z_1^7*z_0^3 + 2592*z_2^2*z_1^4*z_0^4 - 5184*z_2*z_1^5*z_0^4 + 2592*z_1^6*z_0^4 - 1728*z_2^3*z_1^2*z_0^5 + 5184*z_2^2*z_1^3*z_0^5 - 5184*z_2*z_1^4*z_0^5 + 1728*z_1^5*z_0^5 + 432*z_2^4*z_0^6 - 1728*z_2^3*z_1*z_0^6 + 2592*z_2^2*z_1^2*z_0^6 - 1728*z_2*z_1^3*z_0^6 + 432*z_1^4*z_0^6 - 784*z_1^9 + 3264*z_2*z_1^7*z_0 - 3168*z_1^8*z_0 - 4464*z_2^2*z_1^5*z_0^2 - 624*z_3*z_1^6*z_0^2 + 10272*z_2*z_1^6*z_0^2 - 6624*z_1^7*z_0^2 + 2016*z_2^3*z_1^3*z_0^3 + 1536*z_3*z_2*z_1^4*z_0^3 - 10224*z_2^2*z_1^4*z_0^3 - 864*z_3*z_1^5*z_0^3 + 15168*z_2*z_1^5*z_0^3 - 8352*z_1^6*z_0^3 - 720*z_3*z_2^2*z_1^2*z_0^4 + 2592*z_2^3*z_1^2*z_0^4 - 288*z_3^2*z_1^3*z_0^4 + 1632*z_3*z_2*z_1^3*z_0^4 - 9360*z_2^2*z_1^3*z_0^4 - 336*z_3*z_1^4*z_0^4 + 12912*z_2*z_1^4*z_0^4 - 6432*z_1^5*z_0^4 - 288*z_3*z_2^3*z_0^5 + 432*z_2^4*z_0^5 + 384*z_3^2*z_2*z_1*z_0^5 - 576*z_3*z_2^2*z_1*z_0^5 + 864*z_2^3*z_1*z_0^5 - 288*z_3^2*z_1^2*z_0^5 + 672*z_3*z_2*z_1^2*z_0^5 - 5040*z_2^2*z_1^2*z_0^5 + 6720*z_2*z_1^3*z_0^5 - 2880*z_1^4*z_0^5 - 32*z_3^3*z_0^6 + 192*z_3^2*z_2*z_0^6 - 288*z_3*z_2^2*z_0^6 + 720*z_2^3*z_0^6 - 96*z_3^2*z_1*z_0^6 + 192*z_3*z_2*z_1*z_0^6 - 1872*z_2^2*z_1*z_0^6 + 1776*z_2*z_1^2*z_0^6 - 592*z_1^3*z_0^6 - 1320*z_2^2*z_1^6 + 1120*z_3*z_1^7 - 432*z_2*z_1^7 + 3792*z_1^8 + 3480*z_2^3*z_1^4*z_0 - 2928*z_3*z_2*z_1^5*z_0 - 1368*z_2^2*z_1^5*z_0 + 1568*z_3*z_1^6*z_0 - 11104*z_2*z_1^6*z_0 + 11392*z_1^7*z_0 - 2232*z_2^4*z_1^2*z_0^2 + 1008*z_3*z_2^2*z_1^3*z_0^2 + 3792*z_2^3*z_1^3*z_0^2 + 784*z_3^2*z_1^4*z_0^2 - 3120*z_3*z_2*z_1^4*z_0^2 + 8172*z_2^2*z_1^4*z_0^2 + 2312*z_3*z_1^5*z_0^2 - 25632*z_2*z_1^5*z_0^2 + 17256*z_1^6*z_0^2 - 72*z_2^5*z_0^3 + 1200*z_3*z_2^3*z_1*z_0^3 - 1656*z_2^4*z_1*z_0^3 - 1104*z_3^2*z_2*z_1^2*z_0^3 + 864*z_3*z_2^2*z_1^2*z_0^3 - 1488*z_2^3*z_1^2*z_0^3 + 752*z_3^2*z_1^3*z_0^3 - 3808*z_3*z_2*z_1^3*z_0^3 + 18288*z_2^2*z_1^3*z_0^3 + 1568*z_3*z_1^4*z_0^3 - 28504*z_2*z_1^4*z_0^3 + 14360*z_1^5*z_0^3 - 48*z_3^2*z_2^2*z_0^4 + 96*z_3*z_2^3*z_0^4 + 972*z_2^4*z_0^4 + 112*z_3^3*z_1*z_0^4 - 480*z_3^2*z_2*z_1*z_0^4 + 648*z_3*z_2^2*z_1*z_0^4 - 5064*z_2^3*z_1*z_0^4 + 712*z_3^2*z_1^2*z_0^4 - 2112*z_3*z_2*z_1^2*z_0^4 + 13152*z_2^2*z_1^2*z_0^4 + 624*z_3*z_1^3*z_0^4 - 14176*z_2*z_1^3*z_0^4 + 6364*z_1^4*z_0^4 - 16*z_3^3*z_0^5 - 64*z_3^2*z_2*z_0^5 + 96*z_3*z_2^2*z_0^5 - 240*z_2^3*z_0^5 + 192*z_3^2*z_1*z_0^5 - 704*z_3*z_2*z_1*z_0^5 + 1824*z_2^2*z_1*z_0^5 - 2752*z_2*z_1^2*z_0^5 + 1664*z_1^3*z_0^5 + 100*z_3^2*z_0^6 - 200*z_3*z_2*z_0^6 + 500*z_2^2*z_0^6 - 800*z_2*z_1*z_0^6 + 400*z_1^2*z_0^6 - 732*z_2^4*z_1^3 + 1180*z_3*z_2^2*z_1^4 - 36*z_2^3*z_1^4 - 456*z_3^2*z_1^5 - 144*z_3*z_2*z_1^5 + 3708*z_2^2*z_1^5 - 3816*z_3*z_1^6 + 2912*z_2*z_1^6 - 8936*z_1^7 + 960*z_2^5*z_1*z_0 - 1584*z_3*z_2^3*z_1^2*z_0 - 468*z_2^4*z_1^2*z_0 + 624*z_3^2*z_2*z_1^3*z_0 + 1040*z_3*z_2^2*z_1^3*z_0 - 5344*z_2^3*z_1^3*z_0 - 376*z_3^2*z_1^4*z_0 + 6720*z_3*z_2*z_1^4*z_0 - 2600*z_2^2*z_1^4*z_0 - 3920*z_3*z_1^5*z_0 + 22376*z_2*z_1^5*z_0 - 18648*z_1^6*z_0 - 168*z_3*z_2^4*z_0^2 + 156*z_2^5*z_0^2 + 308*z_3^2*z_2^2*z_1*z_0^2 - 312*z_3*z_2^3*z_1*z_0^2 + 504*z_2^4*z_1*z_0^2 - 116*z_3^3*z_1^2*z_0^2 + 24*z_3^2*z_2*z_1^2*z_0^2 - 152*z_3*z_2^2*z_1^2*z_0^2 - 660*z_2^3*z_1^2*z_0^2 - 1928*z_3^2*z_1^3*z_0^2 + 5856*z_3*z_2*z_1^3*z_0^2 - 14072*z_2^2*z_1^3*z_0^2 - 3724*z_3*z_1^4*z_0^2 + 30524*z_2*z_1^4*z_0^2 - 19840*z_1^5*z_0^2 - 16*z_3^3*z_2*z_0^3 + 76*z_3^2*z_2^2*z_0^3 - 896*z_3*z_2^3*z_0^3 + 1152*z_2^4*z_0^3 + 56*z_3^3*z_1*z_0^3 + 976*z_3^2*z_2*z_1*z_0^3 - 872*z_3*z_2^2*z_1*z_0^3 + 712*z_2^3*z_1*z_0^3 - 1136*z_3^2*z_1^2*z_0^3 + 4088*z_3*z_2*z_1^2*z_0^3 - 13460*z_2^2*z_1^2*z_0^3 - 672*z_3*z_1^3*z_0^3 + 21808*z_2*z_1^3*z_0^3 - 11816*z_1^4*z_0^3 - 92*z_3^3*z_0^4 + 492*z_3^2*z_2*z_0^4 - 548*z_3*z_2^2*z_0^4 + 1720*z_2^3*z_0^4 - 776*z_3^2*z_1*z_0^4 + 1392*z_3*z_2*z_1*z_0^4 - 6612*z_2^2*z_1*z_0^4 - 400*z_3*z_1^2*z_0^4 + 8476*z_2*z_1^2*z_0^4 - 3652*z_1^3*z_0^4 - 125*z_2^6 + 280*z_3*z_2^4*z_1 + 90*z_2^5*z_1 - 194*z_3^2*z_2^2*z_1^2 - 240*z_3*z_2^3*z_1^2 + 1074*z_2^4*z_1^2 + 40*z_3^3*z

## **3. Applications**

### 3.1 Equations for commuting operators

In [AAM-2020 120, 102065](https://www.sciencedirect.com/science/article/abs/pii/S0196885820300683) the authors study how to compute commuting operators for a linear differential operator $L = \partial^2 + u$ for an arbitrary differential function $u$. More precisely, given an operator
$$P_m = \partial^m + p_{m,2}\partial^{m-2} + \ldots + p_{m,m-1}\partial + p_{m,m},$$
what are the values for $u$ and $p_{m,*}$ to obtain an operator $P_m$ that commute with $L$, i.e., $[L,P] = LP_m - P_mL = 0$.

This is a problem we can solve using this software. As a demonstration, let us fix $m = 5$. We can then create the operators $L$ and $P$ generically:

In [27]:
B = DifferentialRing(QQ)
R.<p2,p3,p4,p5,u,z> = DPolynomialRing(B)
L = z[2] + u*z
P = z[5] + p2*z[3] + p3*z[2] + p4*z[1] + p5*z
show("L:", L)
show("P:", P)

'L:' u_0*z_0 + z_2

'P:' p2_0*z_3 + p3_0*z_2 + p4_0*z_1 + p5_0*z_0 + z_5

The commutator $[L,P]$ can be computed by evaluating the polynomials:

In [29]:
C = L(z=P) - P(z=L)

The coefficients can be get for different orders of $z$:

In [31]:
for i in range(C.order(z)+1):
    show(f"coefficient of z^({i}):", C.coefficient(z[i]))

'coefficient of z^(0):' -p2_0*u_3 - p3_0*u_2 - p4_0*u_1 + p5_2 - u_5

'coefficient of z^(1):' (-3)*p2_0*u_2 + (-2)*p3_0*u_1 + p4_2 + 2*p5_1 + (-5)*u_4

'coefficient of z^(2):' (-3)*p2_0*u_1 + p3_2 + 2*p4_1 + (-10)*u_3

'coefficient of z^(3):' p2_2 + 2*p3_1 + (-10)*u_2

'coefficient of z^(4):' 2*p2_1 + (-5)*u_1

We can observe from this set of coefficients that we can compute the values for $p_2,p_3,p_4,p_5$ from the for last coefficients. We can solve the system using the class `DSystem`:

In [33]:
system = DSystem([C.coefficient(z[i]) for i in range(1, C.order(z)+1)], variables=[p2,p3,p4,p5])
system

System over [Ring of operator polynomials in (p2, p3, p4, p5, u, z) over Differential Ring [[Rational Field], (0,)]] with variables [(p2_*, p3_*, p4_*, p5_*)]:
{
	(-3)*p2_0*u_2 + (-2)*p3_0*u_1 + p4_2 + 2*p5_1 + (-5)*u_4 == 0
	(-3)*p2_0*u_1 + p3_2 + 2*p4_1 + (-10)*u_3 == 0
	p2_2 + 2*p3_1 + (-10)*u_2 == 0
	2*p2_1 + (-5)*u_1 == 0
}

In [35]:
sols = system.solve_linear()
sols

{p2_*: 5/2*u_0,
 p3_*: 15/4*u_1,
 p4_*: 15/8*u_0^2 + 25/8*u_2,
 p5_*: 15/8*u_1*u_0 + 15/16*u_3}

Hence, the value for $P$ is fixed when given the value of $u$. However, we have no guarantee that $L$ and $P$ commute. In fact, there is one remaining equation that provides a condition for $u$:

In [37]:
equation_for_u = C.coefficient(z[0])(dic=sols)
equation_for_u

(-15/8)*u_1*u_0^2 + (-5/8)*u_3*u_0 + (-5/4)*u_2*u_1 + (-1/16)*u_5

Solving this equation is a **hard** problem (in general). But for this specific case, we can find a rational solution in $\mathbb{Q}(x)$:

In [41]:
x = DQ_x('x')
solution = -2/x^2
show("solution: ", solution)
equation_for_u(u=solution)

'solution: ' (-2)/x^2

0

We could now build the corresponding specialized version of $L$ and $P$:

In [55]:
L_sub = L(u=solution)
P_sub = P(dic=sols)(u=solution)
show(LatexExpr(r"\tilde{L} = "), L_sub)
show(LatexExpr(r"\tilde{P} = "), P_sub)
show(LatexExpr(r"[\tilde{L}, \tilde{P}] = "), L_sub(z=P_sub) - P_sub(z=L_sub))

\tilde{L} =  z_2 + ((-2)/x^2)*z_0

\tilde{P} =  z_5 + ((-5)/x^2)*z_3 + 15/x^3*z_2 + ((-30)/x^4)*z_1 + 30/x^5*z_0

[\tilde{L}, \tilde{P}] =  0

#### Generic implementation

A partial problem can be solved for arbitrary orders of $L$ and $P_m$ can be solved within the software using the method `almost_commuting_wilson` in `dalgebra.hierarchies`.

In [22]:
from dalgebra.hierarchies import almost_commuting_wilson
almost_commuting_wilson?

Signature:     
almost_commuting_wilson(
    n: int,
    m: int,
    name_u: str = 'u',
    name_z: str = 'z',
    method='diff',
)
Docstring:     
   Method to compute an element on the almost-commuting basis.

   Let L be a linear differential operator of order n. We say that A
   *almost commute with `L`* if ord([L,A]) <= n-2. In general, the
   commutator will have order n+m-1 where m is the order of A. Then,
   if the commutator has such a low order, we say that it *almost
   commute*.

   It is easy to see that:

   * A almost commuting with L does not imply that L almost commute
     with A.

   * Let AC(L) the set of almost commuting linear operators. Then
     AC(L) is a C-vector space where C is the field of constants of
     the differential ring R where L and A are built over.

   It was shown by Wilson (TODO: add reference), that for a
   Schrödinger type operator L of order n

      L_n = \partial^n + u_{n-2}\partial^{n-2} + ... + u_1\partial
      + u_0,

   there is a u

### 3.2 Parameter identifiability

In Parameter Identifiability problems we need to consider when, given an output function of a dynamical system, we can identify the values (at least with finitely many possibilities) of the parameters of the system. This problem is in the end reduced to obtain an equation only involving parameters and the output variables and checking the coefficients of these equations.

Let us consider the following dynamical system:
$$\left\{\begin{array}{rl}
    C(n+1) + K_{se}C(n)S(n) + (K_{cp} + K_{rb} - 1)C(n) - I_0K_{se}S(n) & = 0\\
    S(n+1) - K_{sc}C(n)S(n) - K_{rb}C(n) + (I_0K_{se} - 1)S(n) & = 0
\end{array}\right.$$

where we have two difference equations on the variables $C(n), S(n)$ and the parameters $K_{se}, K_{cp}, K_{rb}, I_0, K_{sc}$.

Let us suppose that we can obtain the values for $Y(n) = C(n) + S(n)$, which parameters are identifiable?

To solve this problem we first need to cthe the appropriate difference setting where we have to create the corresponding parameters and the the difference polynomial ring in the variables $C(n), S(n), Y(n)$:

In [2]:
B.<I_0, K_cp, K_rb, K_sc, K_se> = QQ[]
SB = DifferenceRing(B)
show("base difference ring: ", SB)
R.<C,S,Y> = DPolynomialRing(SB)
show("difference polynomial ring:", R)

'base difference ring: ' Difference Ring [[Multivariate Polynomial Ring in I_0, K_cp, K_rb, K_sc, K_se over Rational Field], (Id,)]

'difference polynomial ring:' Ring of operator polynomials in (C, S, Y) over Difference Ring [[Multivariate Polynomial Ring in I_0, K_cp, K_rb, K_sc, K_se over Rational Field], (Id,)]

Then, we need to create the system described above that also sets the identity $Y(n) = C(n) + S(n)$. Since we want then to eliminate the variables $S(n)$ and $C(n)$, we set them as variables of the system:

In [3]:
equation1 = C[1] + K_se*C*S + (K_cp + K_rb - 1)*C - I_0*K_se*S
equation2 = S[1] - K_sc*C*S - K_rb*C + (I_0*K_se - 1)*S
equation3 = Y - C - S
system = DSystem([equation1, equation2, equation3], variables=[S,C])
system

System over [Ring of operator polynomials in (C, S, Y) over Difference Ring [[Multivariate Polynomial Ring in I_0, K_cp, K_rb, K_sc, K_se over Rational Field], (Id,)]] with variables [(S_*, C_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
	-C_0 - S_0 + Y_0 == 0
}

The differential resultant allows to compute this elimination:

In [4]:
%%time
res = system.diff_resultant()

CPU times: user 2.6 s, sys: 927 µs, total: 2.6 s
Wall time: 2.61 s


In [6]:
res.order(Y), res.degree(), len(res.monomials())

(2, 4, 21)

And now we are ready to start analyzing the coefficients of this resultant.

In [7]:
mons = res.monomials(); coeffs = res.coefficients()
for i in range(len(mons)):
    print(f"{mons[i]}\t--> (degree {coeffs[i].wrapped.degree()})\t{str(coeffs[i])[:30]})")

Y_1^4	--> (degree 6)	K_sc^4*K_se^2 - 2*K_sc^3*K_se^)
Y_1^3*Y_0	--> (degree 8)	-I_0*K_sc^5*K_se^2 + 2*I_0*K_s)
Y_1^2*Y_0^2	--> (degree 10)	I_0^2*K_sc^5*K_se^3 - 4*I_0^2*)
Y_1*Y_0^3	--> (degree 11)	I_0^2*K_cp*K_sc^6*K_se^2 + I_0)
Y_0^4	--> (degree 12)	I_0^2*K_cp^2*K_sc^6*K_se^2 + 2)
Y_2*Y_1^2	--> (degree 5)	2*K_sc^4*K_se - 6*K_sc^3*K_se^)
Y_2*Y_1*Y_0	--> (degree 7)	-I_0*K_sc^5*K_se + 3*I_0*K_sc^)
Y_2*Y_0^2	--> (degree 9)	I_0^2*K_sc^5*K_se^2 - 5*I_0^2*)
Y_1^3	--> (degree 9)	I_0^2*K_sc^5*K_se^2 - 3*I_0^2*)
Y_1^2*Y_0	--> (degree 11)	-I_0^3*K_sc^5*K_se^3 + 3*I_0^3)
Y_1*Y_0^2	--> (degree 12)	-3*I_0^3*K_cp*K_sc^5*K_se^3 - )
Y_0^3	--> (degree 13)	-2*I_0^3*K_cp^2*K_sc^5*K_se^3 )
Y_2^2	--> (degree 4)	K_sc^4 - 4*K_sc^3*K_se + 6*K_s)
Y_2*Y_1	--> (degree 8)	-2*I_0^2*K_sc^4*K_se^2 + 8*I_0)
Y_2*Y_0	--> (degree 8)	-2*I_0*K_cp^2*K_sc^4*K_se - 2*)
Y_1^2	--> (degree 12)	I_0^4*K_sc^4*K_se^4 - 4*I_0^4*)
Y_1*Y_0	--> (degree 13)	2*I_0^4*K_cp*K_sc^4*K_se^4 - 8)
Y_0^2	--> (degree 14)	I_0^4*K_cp^2*K_sc^4*K_se^4 